In [1]:
import os
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
from autogen.retrieve_utils import TEXT_FORMATS

config_list = [{"model": "gpt-3.5-turbo-0125", "api_key": os.environ["OPENAI_API_KEY"], "api_type": "openai"}]
assert len(config_list) > 0
print("models to use: ", [config_list[i]["model"] for i in range(len(config_list))])

models to use:  ['gpt-3.5-turbo-0125']


In [3]:
MONGODB_CONFIG = {
    "connection_string": os.environ["MONGODB_URI"] ,
    "database_name": "vector_mflix",  # Target database
    "collection_name": "movies_with_vectors",  # Collection with movie embeddings
    "index_name": "vector_index",  # Index name for vector similarity search
    "wait_until_index_ready": 120.0,  # Seconds to wait for the index to be ready
    "wait_until_document_ready": 120.0,  # Seconds to wait for documents to be indexed
}

In [4]:
# Load the embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
# Create the AssistantAgent
assistant = AssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant for retrieving movie information based on a user's plot description.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": [
            {"model": "gpt-4", "temperature": 0.7}  # Update with your desired model config
        ],
    },
)

In [6]:
# Create the RetrieveUserProxyAgent
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "default",
        "vector_db": "mongodb",  # MongoDB Atlas database
        "collection_name": MONGODB_CONFIG["collection_name"],
        "db_config": MONGODB_CONFIG,
        "get_or_create": True,  # Create collection/index if not already present
        "overwrite": False,  # Avoid overwriting existing documents and indexes
        "chunk_token_size": 2000,
    },
    code_execution_config=False,  # No code execution in this scenario
)

2024-12-07 10:22:01,030 - autogen.agentchat.contrib.retrieve_user_proxy_agent - WARNING - docs_path is not provided in retrieve_config. Will raise ValueError if the collection `movies_with_vectors` doesn't exist. Set docs_path to None to suppress this warning.


In [ ]:
# reset the assistant. Always reset the assistant before starting a new conversation.
assistant.reset()

# given a problem, we use the ragproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the ragproxyagent for processing.
# The conversation continues until the termination condition is met, in RetrieveChat, the termination condition when no human-in-loop is no code block detected.
# With human-in-loop, the conversation will continue until the user says "exit".
movie_problem = "A group of bandits stage a brazen train hold-up, only to find a determined posse hot on their heels."
chat_result = ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem=movie_problem)

In [7]:

# Reset the assistant before starting a conversation
assistant.reset()

# Example user problem: Finding a movie by plot
plot_description = "The cartoonist, Winsor McCay, brings the Dinosaurus back to life in the figure of his latest creation, Gertie the Dinosaur."

# Define the process to handle the chat
def find_movie_by_plot(description):
    # Generate an embedding for the plot using the embedding model
    plot_embedding = embedding_model.encode(description).tolist()

    # Construct a problem for the RAG proxy agent
    problem = {
        "task": "Find a movie based on plot similarity",
        "plot_embedding": plot_embedding,
    }

    # Start a conversation between the assistant and the RAG proxy agent
    chat_result = ragproxyagent.initiate_chat(
        assistant, message=ragproxyagent.message_generator, problem=problem
    )

    # Return the chat result
    return chat_result

# Find the movie based on the provided plot description
result = find_movie_by_plot(plot_description)
print("Chat Result:")
print(result)


VectorDB returns doc_ids:  [['7968cf3c', 'bdfbc921']]
Adding content of doc 7968cf3c to context.
Adding content of doc bdfbc921 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

If user's intent is question answering, you must give as short an answer as possi